In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
import pandahouse
from datetime import datetime, timedelta

In [2]:
# Подключаемся к БД
connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20220420'
}

## 1. В feed_actions для каждого юзера посчитаем число просмотров и лайков контента. В message_actions для каждого юзера считаем, сколько он получает и отсылает сообщений, скольким людям он пишет, сколько людей пишут ему.

In [3]:
q = '''
    SELECT
        user_id,
        toDate(time) as event_date,
        MIN(gender) as gender,
        MIN(age) as age,
        MIN(os) as os, 
        countIf(action='view') as views, 
        countIf(action='like') as likes
    FROM
        simulator_20220420.feed_actions
    WHERE
        toDate(time) = yesterday()
    GROUP BY
        user_id, event_date
    '''

feed_df = pandahouse.read_clickhouse(q, connection=connection)
feed_df

,user_id,event_date,gender,age,os,views,likes
0,61279,2022-05-17,1,32,iOS,70,25
1,74961,2022-05-17,1,33,iOS,33,8
2,74851,2022-05-17,0,28,Android,35,7
3,144417,2022-05-17,0,14,iOS,27,7
4,107325,2022-05-17,0,16,Android,30,7
...,...,...,...,...,...,...,...
19041,30254,2022-05-17,1,31,Android,20,4
19042,166975,2022-05-17,0,32,Android,57,8
19043,141838,2022-05-17,1,18,iOS,18,6
19044,167401,2022-05-17,1,15,Android,87,24


In [4]:
q = '''
    -- В message_actions для каждого юзера считаем, сколько он получает и отсылает сообщений, скольким людям он пишет, сколько людей пишут ему.
    SELECT 
        CASE
            WHEN reciever_id = 0 THEN user_id
            WHEN user_id = 0 THEN reciever_id
            ELSE user_id
        END as user_id,
        yesterday() as event_date,
        messages_received,
        messages_sent,
        users_received,
        users_sent
    FROM
        -- Для каждого юзера посчитаем, сколько он отсылает сообщений и скольким людям он пишет
        (
        SELECT
            user_id,
            toDate(time) as event_date,
            COUNT(reciever_id) as messages_sent,
            uniqExact(reciever_id) as users_sent
        FROM
            simulator_20220420.message_actions
        WHERE
            toDate(time) = yesterday()
        GROUP BY
            user_id,
            event_date
        ) sent

        FULL JOIN

        -- Для каждого юзера посчитаем, сколько он получает сообщений и сколько людей пишут ему
        (
        SELECT
            reciever_id,
            toDate(time) as event_date,
            COUNT(user_id) as messages_received,
            uniqExact(user_id) as users_received
        FROM
            simulator_20220420.message_actions
        WHERE
            toDate(time) = yesterday()
        GROUP BY
            reciever_id,
            event_date
        ) received

        ON sent.user_id = received.reciever_id
    '''

messenger_df_0 = pandahouse.read_clickhouse(q, connection=connection)
messenger_df_0

,user_id,event_date,messages_received,messages_sent,users_received,users_sent
0,20984,2022-05-17,0,5,0,5
1,15004,2022-05-17,0,5,0,2
2,120847,2022-05-17,12,9,11,9
3,107243,2022-05-17,0,4,0,4
4,5570,2022-05-17,3,4,3,4
...,...,...,...,...,...,...
1899,118317,2022-05-17,2,0,2,0
1900,114160,2022-05-17,5,0,5,0
1901,118026,2022-05-17,2,0,2,0
1902,117823,2022-05-17,3,0,3,0


In [5]:
q = '''
    SELECT
        id as user_id,
        MIN(gender) as gender, 
        MIN(age) as age, 
        MIN(os) as os
    FROM
        (SELECT
            user_id as id,
            MIN(gender) as gender,
            MIN(age) as age,
            MIN(os) as os
        FROM
            simulator_20220420.message_actions
        WHERE
            toDate(time) = yesterday()
        GROUP BY
            id

        UNION ALL

        SELECT
            reciever_id as id,
            MIN(gender) as gender,
            MIN(age) as age,
            MIN(os) as os
        FROM
            simulator_20220420.message_actions
        WHERE
            toDate(time) = yesterday()
        GROUP BY
            id)
    GROUP BY
        user_id
    '''
messenger_attributes_df = pandahouse.read_clickhouse(q, connection=connection)
messenger_attributes_df

,user_id,gender,age,os
0,107358,0,20,Android
1,108391,1,40,Android
2,20891,0,20,Android
3,124104,0,17,Android
4,125960,0,20,Android
...,...,...,...,...
1899,127527,0,15,Android
1900,125443,0,50,iOS
1901,9652,1,15,Android
1902,123669,1,20,Android


In [6]:
# Объединение таблиц messenger
messenger_df = pd.merge(messenger_df_0, messenger_attributes_df, on='user_id', how='inner')
messenger_df

,user_id,event_date,messages_received,messages_sent,users_received,users_sent,gender,age,os
0,20984,2022-05-17,0,5,0,5,1,28,iOS
1,15004,2022-05-17,0,5,0,2,0,52,iOS
2,120847,2022-05-17,12,9,11,9,0,17,Android
3,107243,2022-05-17,0,4,0,4,1,27,Android
4,5570,2022-05-17,3,4,3,4,0,18,Android
...,...,...,...,...,...,...,...,...,...
1899,118317,2022-05-17,2,0,2,0,0,19,Android
1900,114160,2022-05-17,5,0,5,0,0,18,Android
1901,118026,2022-05-17,2,0,2,0,0,19,Android
1902,117823,2022-05-17,3,0,3,0,0,16,Android


## 2. Далее объединяем две таблицы в одну.

In [7]:
# Объединяем feed и messenger
full_df = pd.merge(feed_df, messenger_df, on=['user_id', 'event_date', 'gender', 'age', 'os'], how='outer').fillna(0)
full_df

,user_id,event_date,gender,age,os,views,likes,messages_received,messages_sent,users_received,users_sent
0,61279,2022-05-17,1,32,iOS,70.0,25.0,0.0,0.0,0.0,0.0
1,74961,2022-05-17,1,33,iOS,33.0,8.0,0.0,0.0,0.0,0.0
2,74851,2022-05-17,0,28,Android,35.0,7.0,0.0,0.0,0.0,0.0
3,144417,2022-05-17,0,14,iOS,27.0,7.0,0.0,0.0,0.0,0.0
4,107325,2022-05-17,0,16,Android,30.0,7.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
20865,118317,2022-05-17,0,19,Android,0.0,0.0,2.0,0.0,2.0,0.0
20866,114160,2022-05-17,0,18,Android,0.0,0.0,5.0,0.0,5.0,0.0
20867,118026,2022-05-17,0,19,Android,0.0,0.0,2.0,0.0,2.0,0.0
20868,117823,2022-05-17,0,16,Android,0.0,0.0,3.0,0.0,3.0,0.0


In [8]:
# Предобработка full_df
full_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20870 entries, 0 to 20869
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   user_id            20870 non-null  uint32        
 1   event_date         20870 non-null  datetime64[ns]
 2   gender             20870 non-null  int8          
 3   age                20870 non-null  int16         
 4   os                 20870 non-null  object        
 5   views              20870 non-null  float64       
 6   likes              20870 non-null  float64       
 7   messages_received  20870 non-null  float64       
 8   messages_sent      20870 non-null  float64       
 9   users_received     20870 non-null  float64       
 10  users_sent         20870 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int16(1), int8(1), object(1), uint32(1)
memory usage: 1.6+ MB


In [9]:
# Предобработка full_df
# Заменим float значения на int
float_cols = ['views', 'likes', 'messages_received', 'messages_sent', 'users_received', 'users_sent']
full_df[float_cols] = full_df[float_cols].astype(int)

# Заменим значения 0 и 1 в столбце gender на более понятные male и female
full_df.loc[full_df['gender']==0, 'gender'] = 'male'
full_df.loc[full_df['gender']==1, 'gender'] = 'female'

# Разобъем значения age на bins для удобства 
age_min = full_df['age'].min()
age_max = full_df['age'].max()

age_groups = [f'{age_min}-18', '19-25', '26-45', '46-65', '66+']
age_bins = [age_min, 18, 25, 45, 65, age_max]
full_df['age_bins'] = pd.cut(full_df['age'], 
                             bins=age_bins, 
                             labels=age_groups)

full_df

,user_id,event_date,gender,age,os,views,likes,messages_received,messages_sent,users_received,users_sent,age_bins
0,61279,2022-05-17,female,32,iOS,70,25,0,0,0,0,26-45
1,74961,2022-05-17,female,33,iOS,33,8,0,0,0,0,26-45
2,74851,2022-05-17,male,28,Android,35,7,0,0,0,0,26-45
3,144417,2022-05-17,male,14,iOS,27,7,0,0,0,0,NaN
4,107325,2022-05-17,male,16,Android,30,7,0,0,0,0,14-18
...,...,...,...,...,...,...,...,...,...,...,...,...
20865,118317,2022-05-17,male,19,Android,0,0,2,0,2,0,19-25
20866,114160,2022-05-17,male,18,Android,0,0,5,0,5,0,14-18
20867,118026,2022-05-17,male,19,Android,0,0,2,0,2,0,19-25
20868,117823,2022-05-17,male,16,Android,0,0,3,0,3,0,14-18


## 3. Для общей таблицы считаем все эти метрики в разрезе по полу, возрасту и ос. Делаем три разных таска на каждый срез.

In [10]:
# Считаем метрики по полу
metrics = ['gender','event_date','views','likes','messages_received','messages_sent','users_received','users_sent']
metrics_by_gender = full_df[metrics].groupby(['gender','event_date'], as_index=[True, False]).sum()
metrics_by_gender

,,views,likes,messages_received,messages_sent,users_received,users_sent
gender,event_date,,,,,,
female,2022-05-17,399229,85579,206,2959,194,1833
male,2022-05-17,314940,67433,12685,9932,9683,8044


In [11]:
# Считаем метрики по возрасту
metrics = ['age_bins','event_date','views','likes','messages_received','messages_sent','users_received','users_sent']
metrics_by_age = full_df[metrics].groupby(['age_bins','event_date'], as_index=[True, False]).sum()
metrics_by_age

,,views,likes,messages_received,messages_sent,users_received,users_sent
age_bins,event_date,,,,,,
14-18,2022-05-17,125646,26524,5839,4570,5352,4253
19-25,2022-05-17,259495,55325,3296,4067,2815,3706
26-45,2022-05-17,269468,58318,740,3043,606,1372
46-65,2022-05-17,47155,10141,9,705,5,246
66+,2022-05-17,1700,404,0,18,0,9


In [12]:
# Считаем метрики по ОС
metrics = ['os','event_date','views','likes','messages_received','messages_sent','users_received','users_sent']
metrics_by_os = full_df[metrics].groupby(['os','event_date'], as_index=[True, False]).sum()
metrics_by_os

,,views,likes,messages_received,messages_sent,users_received,users_sent
os,event_date,,,,,,
Android,2022-05-17,462405,98742,12848,11295,9839,8885
iOS,2022-05-17,251764,54270,43,1596,38,992


## 4. Финальные данные со всеми метриками записываем в отдельную таблицу в ClickHouse.

Структура финальной таблицы должна быть такая:
* Дата - event_date
* Пол - gender
* Возраст - age
* Операционная система - os
* Число просмотров - views
* Числой лайков - likes
* Число полученных сообщений - messages_received
* Число отправленных сообщений - messages_sent
* От скольких пользователей получили сообщения - users_received
* Скольким пользователям отправили сообщение - users_sent

In [13]:
# Получаем необходимую таблицу
fin_df = full_df[['event_date', 'gender', 'age_bins', 'os', 'views', 'likes', 'messages_received', 'messages_sent', 'users_received', 'users_sent']]
fin_df = fin_df.groupby(['event_date', 'gender', 'age_bins', 'os'], as_index=False).sum().rename({'age_bins':'age'}, axis=1)
fin_df

,event_date,gender,age,os,views,likes,messages_received,messages_sent,users_received,users_sent
0,2022-05-17,female,14-18,Android,44513,9279,71,366,67,333
1,2022-05-17,female,14-18,iOS,25076,5280,4,154,4,137
2,2022-05-17,female,19-25,Android,95006,20121,50,495,48,443
3,2022-05-17,female,19-25,iOS,49572,10695,0,161,0,156
4,2022-05-17,female,26-45,Android,97803,21308,55,836,51,377
5,2022-05-17,female,26-45,iOS,54091,11811,6,393,5,178
6,2022-05-17,female,46-65,Android,17649,3725,2,273,1,94
7,2022-05-17,female,46-65,iOS,8054,1755,0,153,0,54
8,2022-05-17,female,66+,Android,614,150,0,7,0,3
9,2022-05-17,female,66+,iOS,458,117,0,3,0,2


In [14]:
fin_df.dtypes

event_date           datetime64[ns]
gender                       object
age                        category
os                           object
views                         int64
likes                         int64
messages_received             int64
messages_sent                 int64
users_received                int64
users_sent                    int64
dtype: object

In [15]:
# Переведем значения в совместимый с БД формат
fin_df['event_date'] = fin_df['event_date'].apply(lambda x: datetime.isoformat(x))
fin_df['age'] = fin_df['age'].astype('object')

In [16]:
# Загружаем финальную таблицу в схему test
# Подключаемся к БД
connection_2 = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': '656e2b0c9c',
    'user': 'student-rw',
    'database': 'test'
}

# Создаем таблицу
q = '''
    CREATE TABLE IF NOT EXISTS test.evg_dubrovin 
        (
        event_date datetime,
        gender TEXT,
        age TEXT, 
        os TEXT,
        views INTEGER,
        likes INTEGER,
        messages_received INTEGER,
        messages_sent INTEGER,
        users_received INTEGER,
        users_sent INTEGER
        )
        ENGINE = MergeTree 
        ORDER BY (event_date);
    '''
# Отправляем таблицу в базу данных
pandahouse.execute(connection=connection_2, query=q)
pandahouse.to_clickhouse(df=fin_df, table='evgdubrovin_test', index=False, connection=connection_2)

ClickhouseException: b"Code: 60. DB::Exception: Table test.evgdubrovin_test doesn't exist. (UNKNOWN_TABLE) (version 21.12.2.17 (official build))\n"

In [ ]:
# Объединяем таблицы
df_all = pd.concat([metrics_by_gender, metrics_by_age, metrics_by_os], keys=['gender', 'age', 'os'])
df_all.reset_index(level='event_date', inplace=True)
df_all